In [ ]:
# importing the required libraries
import tensorflow as tf
from tensorflow.keras import models , layers
import matplotlib.pyplot as plt

In [ ]:
# declaring the dataset 
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "data",shuffle=True, image_size=(256,256),batch_size=32
)
class_names = dataset.class_names

In [ ]:
EPOCHS = 50
# we are planning to take 80 percent of data as training , 10 percent as validation and other 10 percent as test
def get_dataset_partitions_tf (ds , train_split =  0.8 , test_split = 0.1 , val_split = 0.1 , shuffle = True , shuffle_size = 10000):
    ds_size=len(ds)
    if shuffle:
        ds = ds.shuffle(shuffle_size,seed=12)

    train_size = int(train_split*ds_size)
    val_size = int (val_split*ds_size)
    train_Ds = ds.take(train_size)
    val_Ds = ds.skip(train_size).take(val_size)
    test_Ds = ds.skip(train_size).skip(val_size)
    return  train_Ds,val_Ds,test_Ds 

In [ ]:
# we cache the training dataset to speed up data loading and shuffle it for better performace and prefetch it so that we can overlap prepocessing 
# and model execution
train_dataset , val_dataset , test_dataset = get_dataset_partitions_tf(dataset)
train_dataset=train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset=val_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset=test_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# making all the images to same size and then normalizing the values
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(256,256),
    layers.Rescaling(1.0/255)
])
# random flipping of images in vertical and horizontal direction and rotation by 20 %
data_augmentation = tf.keras.Sequential([

    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

In [ ]:
input_shape = (32, 256, 256, 3)
n_classes = len(class_names)
# defining the network architecture
model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset,
    batch_size=32,
    validation_data=val_dataset,
    verbose=1,
    epochs=50,
)

In [ ]:
import numpy
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(25, 25))
for images, labels in test_dataset.take(1):
    for i in range(12):
        ax = plt.subplot(4, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

In [ ]:
import os
model.save("model.h5")